# 半梯度TD（Semi-gradient TD）的更新过程

半梯度TD是用于函数近似情况下的强化学习方法，之所以叫半梯度，是因为在更新过程中只考虑了改变权重向量w_t对于估计的影响，而不考虑对目标值的影响（即梯度值）。

在使用函数近似的情况下，我们用参数化函数 V(s,θ) 来近似真实的值函数 v_π(s)，其中 θ 是可调参数向量。

### 半梯度TD更新公式

$$ \theta_{t+1} = \theta_t + \alpha \cdot \delta_t \cdot \nabla_\theta V(s_t,\theta_t) $$

其中:

- θ_t 是时间 t 的参数向量
- α 是学习率
- δ_t 是TD误差：$\delta_t = r_{t+1} + \gamma V(s_{t+1},\theta_t) - V(s_t,\theta_t)$
- $ \nabla_\theta V(s_t,\theta_t)$ 是值函数关于参数的梯度

### 算法实现步骤

1. 初始化参数向量 $ \theta $
2. 对于每个回合（episode）   
   a.初始化状态 $ s $  
   b.对于回合中的每一步，直到终止:  
      i. 选择动作a，观察奖励 $ r $ 和下一状态 $ s^{'} $    
      ii. 计算TD误差 $ \delta = r + \gamma \cdot V(s^{'}, \theta) - V(s, \theta)$  
      iii. 计算梯度: $ \nabla_\theta V(s,\theta)$  
      iv. 更新参数: $ \theta = \theta + \alpha \cdot \delta \cdot \nabla_\theta V(s,\theta) $   
      v. 最终 $ s = s^{'} $ 

### 称为“半梯度”的原因？

它之所以被称为"半梯度"，是因为在计算梯度时，我们只考虑了当前状态价值估计 $ V(s_t,θ_t) $ 的梯度，而没有考虑目标值 $ r_{t+1} + γV(s_{t+1},θ_t) 中 V(s_{t+1},θ_t) $  对 θ 的依赖。这种做法在计算上更简单，但理论上不是真正的梯度下降。

In [1]:
import numpy as np

class SemiGradientTD:
    def __init__(self, feature_dim, alpha=0.1, gamma=0.9):
        self.theta = np.zeros(feature_dim)  # 初始化参数
        self.alpha = alpha                  # 学习率
        self.gamma = gamma                  # 折扣因子
        
    def predict(self, features):
        """线性函数近似 V(s,θ) = θ^T·x(s)"""
        return np.dot(self.theta, features)
    
    def update(self, features_t, reward, features_t_plus_1, done):
        """执行半梯度TD更新"""
        # 当前状态的预测值
        V_t = self.predict(features_t)
        
        # 下一状态的预测值（如果是终止状态则为0）
        V_t_plus_1 = 0 if done else self.predict(features_t_plus_1)
        
        # 计算TD误差
        td_error = reward + self.gamma * V_t_plus_1 - V_t
        
        # 半梯度TD更新：注意线性情况下梯度就是特征向量
        self.theta += self.alpha * td_error * features_t
        
        return td_error

# 使用示例
if __name__ == "__main__":
    # 假设状态特征维度为4
    td_learner = SemiGradientTD(feature_dim=4)
    
    # 模拟一个更新步骤
    current_features = np.array([0.5, 0.2, 0.1, 0.8])
    reward = 1.0
    next_features = np.array([0.4, 0.3, 0.2, 0.7])
    done = False
    
    td_error = td_learner.update(current_features, reward, next_features, done)
    print(f"TD Error: {td_error}")
    print(f"Updated Parameters: {td_learner.theta}")

TD Error: 1.0
Updated Parameters: [0.05 0.02 0.01 0.08]
